In [1]:
import itertools, re
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

In [2]:
Logging.setup_logging()

<Logger PySpice (INFO)>

In [3]:
def empty_ir(title="Candidate"):
    return {"title": title, "nodes": ["in","out","0"], "components": []}

In [4]:
def add_vin_and_load(ir, ac=1.0, Rload=10e3):
    # AC=1 V small-signal source recorded in IR
    ir["components"].append({"kind":"VAC","name":"in", "pos":"in","neg":"0","ac": ac})
    # Tiny "wire" placeholder for in->out path (a rewire anchor for series insertions)
    ir["components"].append({"kind":"R","name":"Rpath","n1":"in","n2":"out","value":1e-6})
    # Output load
    ir["components"].append({"kind":"R","name":"Rload","n1":"out","n2":"0","value": Rload})

In [5]:
def emit_pyspice(ir: dict) -> Circuit:
    """
    Build a PySpice Circuit from IR.
    If AcVoltageSource exists (newer PySpice), use it.
    Otherwise insert a 0 V source which we'll rewrite to 'ac <amp>' in text.
    """
    c = Circuit(ir["title"])
    have_ac_src = hasattr(c, "AcVoltageSource")
    for comp in ir["components"]:
        k = comp["kind"].upper()
        if k == "R":
            c.R(comp["name"], comp["n1"], comp["n2"], float(comp["value"]) @ u_Ohm)
        elif k == "C":
            c.C(comp["name"], comp["n1"], comp["n2"], float(comp["value"]) @ u_F)
        elif k == "L":
            c.L(comp["name"], comp["n1"], comp["n2"], float(comp["value"]) @ u_H)
        elif k == "VAC":
            if have_ac_src:
                c.AcVoltageSource(comp["name"], comp["pos"], comp["neg"],
                                  amplitude=float(comp["ac"]) @ u_V)
            else:
                c.V(comp["name"], comp["pos"], comp["neg"], 0 @ u_V)  # will patch in text
        else:
            raise ValueError(f"Unsupported kind: {k}")
    return c

In [6]:
def add_series(ir, name, kind, value):
    # Make a unique new node label tied to current part count
    next_idx = sum(1 for c in ir["components"] if c["kind"] in ("R","C","L"))
    new_node = f"n{next_idx}"
    # Redirect Rpath end to the new node
    for comp in ir["components"]:
        if comp["name"] == "Rpath":
            comp["n2"] = new_node
            break
    # Place the new element from new_node to 'out'
    ir["components"].append({"kind":kind, "name":name, "n1":new_node, "n2":"out", "value":value})

def add_shunt(ir, name, kind, value, node="out"):
    ir["components"].append({"kind":kind, "name":name, "n1":node, "n2":"0", "value":value})

In [7]:
R_VALUES = [1e3]
C_VALUES = [160e-9]
L_VALUES = [160e-3]

def pick_value(kind, step):
    if kind == "R": return R_VALUES[step % len(R_VALUES)]
    if kind == "C": return C_VALUES[step % len(C_VALUES)]
    if kind == "L": return L_VALUES[step % len(L_VALUES)]
    raise ValueError(kind)

In [8]:
OPS   = ["series","shunt"]
# Allow all part kinds for both ops; tweak if you want stricter RC/LC only
KINDS = {"series":["R","L","C"], "shunt":["R","C","L"]}

def build_candidate(seq_ops, seq_kinds):
    ir = empty_ir("Candidate")
    add_vin_and_load(ir, ac=1.0, Rload=10e3)
    for step, (op, kind) in enumerate(zip(seq_ops, seq_kinds), start=1):
        val = pick_value(kind, step-1)
        if op == "series":
            add_series(ir, f"{kind}{step}", kind, val)
        else:
            add_shunt(ir, f"{kind}{step}", kind, val, node="out")
    return ir

In [10]:
def enumerate_all(max_steps=2):
    all_circuits = []
    for ops in itertools.product(OPS, repeat=max_steps):
        choice_lists = [KINDS[o] for o in ops]
        for kinds in itertools.product(*choice_lists):
            ir  = build_candidate(ops, kinds)
            ckt = emit_pyspice(ir)
            all_circuits.append({"ops":ops, "kinds":kinds, "ir":ir, "circuit":ckt})
    return all_circuits

In [15]:
def save_all_to_one(cands, filename="all_candidates.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Total candidates: {len(cands)}\n\n")
        for i, c in enumerate(cands, 1):
            ops_str   = "->".join(c["ops"])
            kinds_str = " ".join(c["kinds"])
            f.write(f"=== Candidate {i} ===\n")
            f.write(f"ops:   {ops_str}\n")
            f.write(f"kinds: {kinds_str}\n")


            f.write("\n")
    print(f"All candidates written to {filename}")

In [16]:
if __name__ == "__main__":
    cands = enumerate_all(max_steps=2)         # generate everything (no simulation)
    save_all_to_one(cands, "all_candidates.txt")

All candidates written to all_candidates.txt
